In [ ]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd
import re
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

In [2]:
# illness='eating'

In [3]:
# train_control = pd.read_csv('train/control.txt', sep='\t', na_values='na', names=['ID','LABEL','POST'],nrows=30000)
# train_control['CLEAN'] = train_control['POST'].apply(lambda x: re.findall("{'body': (.*?), 'created_utc'", x))

# train_ill1 = pd.read_csv('train/eating.txt', sep='\t', na_values='na', names=['ID','LABEL','POST'],nrows=50000)
# train_ill1['CLEAN'] = train_ill1['POST'].apply(lambda x: re.findall("{'body': (.*?), 'created_utc'", x))
# train_ill2 = pd.read_csv('dev/eating_dev.txt', sep='\t', na_values='na', names=['ID','LABEL','POST'],nrows=10000)
# train_ill2['CLEAN'] = train_ill2['POST'].apply(lambda x: re.findall("{'body': (.*?), 'created_utc'", x))
# train_ill = pd.concat([train_ill1,train_ill2])

# train_df = pd.concat([train_control, train_ill])
# train_df = train_df.sample(frac=1,)
# train_df.reset_index(drop=True,inplace=True)
# train_df['string'] = [''.join(map(str, l)) for l in train_df['CLEAN']]
# train_df = train_df[['LABEL','string']]
# train_df['string'].replace('', np.nan, inplace=True)
# train_df.dropna(inplace=True)
# train_df.columns =['tag','text']

In [4]:
# test_control = pd.read_csv('test/control_test.txt', sep='\t', na_values='na', names=['ID','LABEL','POST'], nrows=10000)
# test_control['CLEAN'] = test_control['POST'].apply(lambda x: re.findall("{'body': (.*?), 'created_utc'", x))

# test_ill = pd.read_csv('test/eating_test.txt', sep='\t', na_values='na', names=['ID','LABEL','POST'],nrows=1)
# test_ill['CLEAN'] = test_ill['POST'].apply(lambda x: re.findall("{'body': (.*?), 'created_utc'", x))

# test_df = test_ill

# test_df = test_df.sample(frac=1,)
# test_df.reset_index(drop=True,inplace=True)
# test_df['string'] = [''.join(map(str, l)) for l in test_df['CLEAN']]
# test_df = test_df[['LABEL','string']]
# test_df['string'].replace('', np.nan, inplace=True)
# test_df.dropna(inplace=True)
# test_df.columns =['tag','text']

In [5]:
# df = pd.concat([train_df,test_df])
# df = df.sample(frac=1)

In [6]:
# def text_cleaner(text):
#     newString = text.lower()
#     newString = re.sub(r'\([^)]*\)', '', newString)
#     newString = re.sub('"','', newString)
#     newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
#     newString = re.sub(r"'s\b","",newString)
#     newString = re.sub("[^a-zA-Z.]", " ", newString)
#     return newString

In [7]:
# contraction_mapping = {"’":"'","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

#                            "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

#                            "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

#                            "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

#                            "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

#                            "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

#                            "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

#                            "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

#                            "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

#                            "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

#                            "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

#                            "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

#                            "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

#                            "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

#                            "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

#                            "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

#                            "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

#                            "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

#                            "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

#                            "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

#                            "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

#                            "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

#                            "you're": "you are", "you've": "you have"}

In [8]:
# df['text'] = df['text'].apply(lambda x: text_cleaner(x))

In [9]:
# df['text'].str.encode('utf-8', 'ignore').str.decode('utf-8')
# df['totalwords'] = df['text'].str.split().str.len()


In [10]:
# def clean(df):

#     df['text'].str.encode('utf-8', 'ignore').str.decode('utf-8')
#     df['totalwords'] = df['text'].str.split().str.len()
#     df = df[~(df['totalwords'] <= 4)]
    
    
#     return df

In [11]:
# df = clean(df)

In [12]:
# def sts(df,illness):

#     sum_tokens = df['totalwords'].sum()
#     means_per_df = df['totalwords'].mean(axis=0)
#     print('Mean per '+illness+ ' group '+' '+ str(means_per_df))
# #     print('Mean per users from the '+ group_char + ' group '+str(mean_per_group))
#     print(illness+' no.of tokens '+ str((sum_tokens)))

In [13]:
# sts_= sts(df,illness)

In [14]:
# df = df[['tag','text']]

In [15]:
# possible_labels = df.tag.unique()

# label_dict = {}
# for index, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = index
# label_dict

In [16]:
# df['label'] = df.tag.replace(label_dict)
# df=df.sample(frac=1)

In [17]:
train_data = pd.read_csv('sexism data/train_split.csv', sep=',', header=None, names=['Quote','Titlu','Source','Topic/ Person','Audience','Commenter Gender','Link','Label 1','Label 2','Label'])

val_data = pd.read_csv('sexism data/test_split.csv', sep=',',header=None, names=['Quote','Titlu','Source','Topic/ Person','Audience','Commenter Gender','Link','Label 1','Label 2','Label'])

train_data = train_data[['Quote', 'Label']]
val_data = val_data[['Quote', 'Label']]
train_data = train_data.iloc[1: , :]
val_data = val_data.iloc[1:, :]

# df = pd.concat([train_data, val_data])

possible_labels = train_data.Label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

train_data['Label'] = train_data.Label.replace(label_dict)
val_data['Label'] = val_data.Label.replace(label_dict)


X= train_data[['Quote']]
y = train_data[['Label']]
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)


skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

for train_index, val_index in skf.split(X, y):
  # splitting Dataframe (dataset not included)
    X_train, X_test = X.iloc[train_index], X.iloc[val_index]
    y_train, y_test = y.iloc[train_index], y.iloc[val_index]
    
X_test = val_data[['Quote']]
y_test = val_data[['Label']]

In [18]:
from transformers import RobertaTokenizer, RobertaForTokenClassification
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


In [19]:
encoded_data_train = tokenizer.batch_encode_plus(
    X_train.Quote.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=200, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_test.Quote.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=200, 
    return_tensors='pt'
)

# train
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.Label.values)

# val
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_test.Label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


In [20]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [21]:
len(dataset_train), len(dataset_val)

(1294, 405)

In [22]:
model = BertForSequenceClassification.from_pretrained("roberta-base",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at roberta-base were not used when initializing BertForSequenceClassification: ['roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.intermediate.dense.bias', 'rober

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['embeddings.word_embeddings.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.LayerNorm.weight', 'embeddings.LayerNorm.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.LayerNorm.weight', 'en

In [23]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [24]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)



In [25]:
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [26]:
from sklearn.metrics import f1_score,precision_score, recall_score, classification_report


def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    class_rep = classification_report(preds_flat,labels_flat,labels=[0,1],output_dict=True)
    df = pd.DataFrame(class_rep).transpose()
    return df
     
    

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [27]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [29]:
print(device)

cpu


In [30]:
def evaluate(dataloader_val):
    
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    #####

    
    #######
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
            
    return loss_val_avg, predictions, true_vals





In [31]:
import torch
torch.cuda.empty_cache()

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    
    model.train()
    train_loss_set = []
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        
        
        outputs = model(**inputs)
        
        
        loss = outputs[0]
        train_loss_set.append(loss.item())
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
        

    
    torch.save(model, 'sexism_det_.pt')   
    
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
     
    
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
#     val_f1_avg = val_f1/ len(dataloader_validation)
#     val_loss_avg = val_loss/ len((dataloader_validation))
    
   
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
model = BertForSequenceClassification.from_pretrained("xlm-roberta-large",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model=torch.load('sexism_det_.pt', map_location=torch.device('cuda'))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
dataloader_validation

In [ ]:
list_=accuracy_per_class(predictions, true_vals)

In [ ]:
df_scores = f1_score_func(predictions,true_vals)

In [ ]:
df_scores.to_csv('sexism_scores_xlm_roberta_multi.csv')